### setup

In [1]:
import os
import argparse
import pickle
import time

import numpy as np; np.seterr(invalid='ignore')
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader

In [2]:
parser = {
    'data_path': '../data/wttsf/',
    'train_file': 'train_1.csv',
    'intermediate_path': '../intermediate/',
    'n_epoch': 4,
    'future': 70,
    'batch_size': 128,
    'hidden_size': 256,
    'log_every': 10,
    'read_from_file': True,
    'train': True,
    'model_name': '',
    'cuda': True,
    'seed': 30082017,
}
args = argparse.Namespace(**parser)

args.cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)

args.intermediate_path = os.path.join(args.intermediate_path, str(args.seed))

### model

In [3]:
class DenseLSTMForecast(nn.Module):
    def __init__(self, hidden_size):
        super(DenseLSTMForecast, self).__init__()
        self.lstm1 = nn.LSTMCell(1, hidden_size, bias=False)
        self.lstm2 = nn.LSTMCell(hidden_size+1, hidden_size, bias=False)
        self.lstm3 = nn.LSTMCell(2*hidden_size+1, hidden_size, bias=False)
        self.linear = nn.Linear(3*hidden_size+1, 1)
        self.relu = nn.ReLU()
        self.hidden_size = hidden_size

    def forward(self, x, future=1):
        o = []
        tt = torch.cuda if args.cuda else torch
        h1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c1_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        h2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c2_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        h3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        c3_t = Variable(tt.FloatTensor(x.size(0), self.hidden_size).zero_())
        
        for x_t in x.chunk(x.size(1), dim=1):
            x_t = x_t.squeeze(dim=1)
            h1_t, c1_t = self.lstm1(x_t, (h1_t, c1_t))
            h1d_t = torch.cat([x_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([x_t, h1_t, h2_t], dim=1)
            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
            h3d_t = torch.cat([x_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h3d_t)
            o.append(o_t)
            
        for i in range(future-1):
            h1_t, c1_t = self.lstm1(o_t, (h1_t, c1_t))
            h1d_t = torch.cat([o_t, h1_t], dim=1)
            h2_t, c2_t = self.lstm2(h1d_t, (h2_t, c2_t))
            h2d_t = torch.cat([o_t, h1_t, h2_t], dim=1)
            h3_t, c3_t = self.lstm3(h2d_t, (h3_t, c3_t))
            h3d_t = torch.cat([o_t, h1_t, h2_t, h3_t], dim=1)
            o_t = self.linear(h3d_t)
            o.append(o_t)

        return torch.stack(o, dim=1)

### utils

In [4]:
def smape(y_pred, y_true):
    raw_smape = np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred))
    kaggle_smape = np.nan_to_num(raw_smape)
    return np.mean(kaggle_smape) * 200

In [5]:
def get_data():
    raw_data_file = os.path.join(args.intermediate_path,
                                 'raw_data.pkl')
    scaled_data_file = os.path.join(args.intermediate_path,
                                    'scaled_data.pkl')
    scaler_file = os.path.join(args.intermediate_path, 'scaler.pkl')
    if not args.read_from_file:
        data_df = pd.read_csv(os.path.join(args.data_path, args.train_file),
                              index_col='Page')
        raw_data = np.nan_to_num(data_df.values.astype('float32'))
        data = np.log1p(raw_data)
        scaler = MaxAbsScaler()
        scaler.fit(np.swapaxes(data[:, :-args.future], 0, 1))
        scaled_data = scaler.transform(np.swapaxes(data, 0, 1))
        scaled_data = np.swapaxes(scaled_data, 0, 1)
        
        with open(raw_data_file, 'wb') as f:
            pickle.dump(raw_data, f)
        with open(scaled_data_file, 'wb') as f:
            pickle.dump(scaled_data, f)
        with open(scaler_file, 'wb') as f:
            pickle.dump(scaler, f)
    else:
        with open(raw_data_file, 'rb') as f:
            raw_data = pickle.load(f)
        with open(scaled_data_file, 'rb') as f:
            scaled_data = pickle.load(f)
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
    return raw_data, scaled_data, scaler

In [6]:
def train(raw_data, scaled_data, scaler, model, criterion, optimizer, epoch):
    print("=> EPOCH {}".format(epoch))
    p = np.random.permutation(raw_data.shape[0])
    inverse_p = np.argsort(p)
    
    input_tensor = torch.from_numpy(scaled_data[p, :-1]).unsqueeze(2)
    target_tensor = torch.from_numpy(scaled_data[p, 1:]).unsqueeze(2)
    dataset = TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    train_loss = 0
    val_output_list = []
    init_time = time.time()
    for i, (inputt, target) in enumerate(data_loader):
        if args.cuda:
            inputt = inputt.cuda()
            target = target.cuda()
        inputt = Variable(inputt)
        target = Variable(target)
        
        output = model(inputt)
        pos = np.random.randint(args.future, raw_data.shape[1]-2*args.future)
        loss = criterion(output[:, pos:pos+args.future],
                         target[:, pos:pos+args.future])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.data[0] * inputt.size(0)
        val_output_list.append(output[:,-args.future:]
                               .data.squeeze(2).cpu().numpy())
        
        if i % args.log_every == 0:
            print("   % Time: {:4.0f}s | Batch: {:4} | Loss: {:.4f}"
                  .format(time.time()-init_time, i+1, loss.data[0]))
        
    val_output_all = np.concatenate(val_output_list, axis=0)[inverse_p]
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(val_output_all, 0, 1)), 0, 1)
    prediction = np.exp(prediction) - 1
    
    train_loss /= raw_data.shape[0]
    val_loss = smape(prediction, raw_data[:, -args.future:])
    
    print("="*10)
    print("   % Epoch: {} | Time: {:4.0f}s | "
          "Train loss: {:.4f} | Val loss: {:.4f}"
          .format(epoch, time.time()-init_time, train_loss , val_loss))
    print("="*10)
    return val_loss

In [7]:
def forecast(scaled_data, scaler, model):
    input_tensor = torch.from_numpy(scaled_data).unsqueeze(2)
    target_tensor = torch.zeros(input_tensor.size(0))
    dataset = torch.utils.data.TensorDataset(input_tensor, target_tensor)
    data_loader = DataLoader(dataset, args.batch_size)
    
    output_list = []
    for inputt, _ in data_loader:
        if args.cuda:
            inputt = inputt.cuda()
        inputt = Variable(inputt)
        output = model(inputt, args.future)
        output_list.append(output.data.squeeze(2).cpu().numpy()
                           [:, -args.future:])
        
    output_all = np.concatenate(output_list, axis=0)
    prediction = np.swapaxes(scaler.inverse_transform(
            np.swapaxes(output_all, 0, 1)), 0, 1)
    prediction = np.exp(prediction) - 1
    prediction[prediction < 0] = 0
    prediction = np.around(prediction)
    return prediction

In [8]:
def save_model(model, epoch, val_loss):
    model_file = os.path.join(args.intermediate_path,
                              "model_seed{}_epoch{}_loss_{:.4f}.pth"
                              .format(args.seed, epoch, val_loss))
    torch.save(model.state_dict(), os.path.join(model_file))

### prepare

In [9]:
raw_data, scaled_data, scaler = get_data()

In [10]:
model = DenseLSTMForecast(args.hidden_size)
if args.cuda:
    model.cuda()
criterion = nn.L1Loss()

In [11]:
optimizer = optim.Adam(model.parameters())
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

### train

In [12]:
if args.train:
    for epoch in range(1, args.n_epoch+1):
        scheduler.step()
        val_loss = train(raw_data, scaled_data, scaler,
                         model, criterion, optimizer, epoch)
        save_model(model, epoch, val_loss)
else:
    model_file = os.path.join(args.intermediate_path, args.model_name)
    model.load_state_dict(torch.load(model_file))

=> EPOCH 1
   % Time:    3s | Batch:    1 | Loss: 0.7329
   % Time:   25s | Batch:   11 | Loss: 0.4946
   % Time:   47s | Batch:   21 | Loss: 0.4393
   % Time:   70s | Batch:   31 | Loss: 0.4425
   % Time:   92s | Batch:   41 | Loss: 0.4393
   % Time:  115s | Batch:   51 | Loss: 0.4688
   % Time:  137s | Batch:   61 | Loss: 0.4569
   % Time:  159s | Batch:   71 | Loss: 0.4631
   % Time:  182s | Batch:   81 | Loss: 0.4673
   % Time:  204s | Batch:   91 | Loss: 0.4480
   % Time:  226s | Batch:  101 | Loss: 0.4528
   % Time:  248s | Batch:  111 | Loss: 0.4227
   % Time:  271s | Batch:  121 | Loss: 0.4448
   % Time:  293s | Batch:  131 | Loss: 0.4460
   % Time:  315s | Batch:  141 | Loss: 0.4196
   % Time:  337s | Batch:  151 | Loss: 0.4434
   % Time:  360s | Batch:  161 | Loss: 0.4278
   % Time:  382s | Batch:  171 | Loss: 0.4168
   % Time:  404s | Batch:  181 | Loss: 0.4547
   % Time:  426s | Batch:  191 | Loss: 0.3765
   % Time:  449s | Batch:  201 | Loss: 0.4443
   % Time:  471s | Batc

KeyboardInterrupt: 

### test

In [ ]:
prediction = forecast(scaled_data, scaler, model)
prediction_file = os.path.join(args.intermediate_path,
                               'prediction_seed{}.pkl'.format(args.seed))

with open(prediction_file, 'wb') as f:
    pickle.dump(prediction, f)